In [1]:
import os
directory = 'bratannotationfiles'
t = 0
loc1=""
loc2=""
# sent.txt file will contain those sentences only which has event phrase
# tuple.txt file will contain event-type;event phrase;companies involved
with open("tuple.txt","w") as fil1:
    with open("sent.txt","w") as fil2:
            for filename in os.listdir(directory):
                if t==0:
                    loc1 = os.path.join(directory, filename)# location of annotated file(.ann)
                    t = 1
                else:
                    t = 0
                    loc2 = os.path.join(directory, filename)# location of .txt file

                    with open(loc1) as f:
                        lines1 = f.readlines()

                    with open(loc2) as f:
                        lines2 = f.readlines()

                    with open(loc2) as f:
                        lines21 = f.read()
                        
                    d1 = {}# creating dictionary with tags related to event phrase as key and list of tags related to company name(involved in that event phrase) as values  
                  
                    d2 = {}# creating dictionary with tags indicating company as keys and corresponding company's name as it's value 
                    
                    d3 = {}# creating dictionary with event type as key and list of  tags related to involved company as values.
                    
                    # building dictionary d1 and d2
                    for i in lines1:

                        s = i.split('\t')
                        s2=s[1].split(' ')

                        if i[0]=='R':

                            s3 = s2[1].split(':')[1]# tag related to event phrase
                            s4 = s2[2].split(':')[1]# tag related to company name

                            s3=s3.strip()
                            s4 = s4.strip()
                            if s3 in d1.keys():
                                 d1[s3].append(s4)
                            else:
                                d1[s3]=[s4]

                        if s2[0]=='Company':
                            d2[s[0]]=s[2].strip()

                    # building dictionary d3
                    for i in lines1:
                        if i[0]=='T': # sentence starting with 'T' in .ann file indicates either event phrase or company name, we'll ignore latter one
                            s = i.split('\t')
                            s2 = s[1].split(' ')
                            s3=s2[1]+','+s2[2]
                            s4 = s2[0].strip()
                            s3 = s3+','+s4        #s4 has event type
                            if s2[0]!='Company':
                                if s3 in d3.keys():
                                    print(s3)
                                    print(loc1)
                                    if s[0] in d1.keys():
                                        d3[s3].append(d1[s[0]])
                                    else:
                                        d3[s3].append(['UNK'])# if no companies are specified for that event,take UNK as its value 
                                else:
                                    if s[0] in d1.keys():
                                        d3[s3]=d1[s[0]]
                                    else:
                                        d3[s3]=['UNK']


                    with open("tuplet.txt","w") as f1:
                        with open("sentt.txt","w") as f2:
                            s = 0
                            e = 0
                            for i in lines2:
                                e = s+len(i)-1

                                t1 = 0
                                l=[]
                                for key,val in d3.items():
                                    s1 = key.split(',')
                                    id1 = int(s1[0])
                                    id2 = int(s1[1])-1
    
                                    if id1>=s and id2<=e:# whether event span belongs to this sentence or not
                                    #if key in i:
                                        if t1==0:
                                            t1=1
                                        else:
                                            l.append('|')

                                        l.append(s1[2]+';')
                                        #evnt = key.split()
                                        l.append(lines21[id1:id2+1]+';')
            #                             l.append(key+';')
                                        l1=val
                                        #print(l1)
                                        for j in l1:
                                            if j=='UNK':
                                                l.append(j)
                                            else:
                                                l.append(d2[j])

                                            l.append(',')
                                            break 
                                        l.pop(-1)
                                if t1==1:

                                    f1.writelines(l)
                                    f1.write('\n')
                                    f2.writelines(i)
                                s = e+1

                    with open("tuplet.txt","r") as f1:
                        with open("sentt.txt","r") as f2:

                            fil1.write(f1.read())
                            fil2.write(f2.read())

                    




In [2]:
# using sent.txt and tuple.txt,we'll create pointer file which will store event-type,starting and ending index of event phrase in a sentence,starting and ending index of company name
# Eg: 
# unused1 Absa , the South African banking group , said its integration with new majority-owner Barclays was proceeding well as it announced a jump in its interim profit .(in sent5.txt)
# Profit 21 27 1 1 (in pointer5.txt file)
s=0
etlist = [] # list of event-types
with open("pointer5.txt","w") as fil3:
    with open("sent5.txt","w") as fil4:
        with open("tuple.txt","r") as fil1:
            with open("sent.txt","r") as fil2:
                f1 = fil1.readlines()
                f2 = fil2.readlines()
                #unu=0
                for i in range(len(f1)):
                    events = f1[i].split('|')
                    f2[i]='unused1 '+f2[i] #adding unused1 token before each sentence   
                    tup1=[]
                    for j in range(len(events)):
                        event = events[j].split(';')
                        et = event[0].strip()
                        if (et) not in etlist:
                            etlist.append(et)
                        es = event[1].split()
                        com = event[2].strip()
                        sent = f2[i].split()
                        l = len(es)
                        tup=[]
                        for k in range(len(sent)):
                            
                            if (sent[k]==es[0] and sent[k+l-1]==es[l-1]) or k+l==len(sent):
                                
                                tup=[et]
                                tup.append(' '+str(k)+' '+str(k+l-1))
                                
                                break
                        
                        if com in f2[i]:                            
                            com = com.split()                          
                            l = len(com)
                            for k in range(len(sent)):
                                #if j+l
                                if (sent[k]==com[0] and sent[k+l-1]==com[l-1]) or k+l==len(sent):
                                    tup.append(' '+str(k)+' '+str(k+l-1))  
                                    tup.append(' | ')
                                     
                                    break
                        else:
                            s=s+1
                            tup.append(' '+'0'+' '+'0')
                            tup.append(' | ')
                        tup1.extend(tup)
                        
                    tup1.pop()
                    fil3.writelines(tup1)
                    fil3.write('\n')
                    fil4.write(f2[i])
print(s/2193)      

0.5795713634290925


In [30]:
with open('event_names.txt','w') as f:
    for e in etlist:
        f.write(e+'\n')